# E-Scooter Availability Analysis

This notebook analyzes whether scooter availability (average distance to nearest scooter) correlates with usage rates across cities.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.neighbors import BallTree
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
from adjustText import adjust_text
from tqdm.auto import tqdm
import os

from tueplots import bundles
from tueplots.constants.color import rgb

import sys
sys.path.insert(0, '..')
from city_info import CITY_POPULATIONS, normalize_city, sanitize_city_name
from noise_detection.non_ride_detection import filter_known_issues

In [ ]:
# Load events
events = pd.read_parquet('../vehicle_events_export.parquet')
events = events[events['city'].notna()].copy()
events['city'] = events['city'].astype('category')

print(f"Total events: {len(events):,}, Cities: {events['city'].nunique()}")

## Build City Demand Hulls

Create polygons representing where people look for scooters:
1. Take all departure positions (where scooters were picked up)
2. Buffer each point by 100m, merge, fill small holes

In [ ]:
# Extract departure positions per city (min 5000 departures for reliable spatial coverage)
city_departures = {}
for city in tqdm(events['city'].cat.categories, desc="Extracting departures"):
    deps = events[(events['city'] == city) & (events['disappeared'] == True)][['lat', 'lon']]
    if len(deps) >= 5000:
        city_departures[city] = deps

print(f"Cities with sufficient departures: {len(city_departures)}")

In [ ]:
# Build demand hulls for visualization
import folium
import pickle

HULL_CHECKPOINT = 'checkpoints/demand_hulls.pkl'
os.makedirs('checkpoints', exist_ok=True)

def fill_small_holes(polygon, min_hole_size_m=100):
    """Remove holes smaller than min_hole_size_m (diameter in meters)."""
    min_area_m2 = np.pi * (min_hole_size_m / 2) ** 2
    min_area_deg2 = min_area_m2 / (111000 ** 2)
    
    if polygon.geom_type == 'Polygon':
        large_holes = [interior for interior in polygon.interiors 
                       if Polygon(interior).area >= min_area_deg2]
        return Polygon(polygon.exterior, large_holes)
    elif polygon.geom_type == 'MultiPolygon':
        return unary_union([fill_small_holes(p, min_hole_size_m) for p in polygon.geoms])
    return polygon

def create_demand_hull(positions, buffer_meters=100, min_hole_size_m=100):
    """Create demand hull polygon from positions with 100m radius buffer."""
    buffer_deg = buffer_meters / 111000
    points = [Point(lon, lat).buffer(buffer_deg) for lat, lon in positions]
    merged = unary_union(points)
    merged = fill_small_holes(merged, min_hole_size_m)
    return merged

if os.path.exists(HULL_CHECKPOINT):
    with open(HULL_CHECKPOINT, 'rb') as f:
        city_demand_hulls = pickle.load(f)
else:
    city_demand_hulls = {}
    for city, deps in tqdm(city_departures.items(), desc="Building demand hulls"):
        positions = deps[['lat', 'lon']].values
        hull = create_demand_hull(positions, buffer_meters=100, min_hole_size_m=100)
        city_demand_hulls[city] = hull
    
    with open(HULL_CHECKPOINT, 'wb') as f:
        pickle.dump(city_demand_hulls, f)

print(f"Demand hulls: {len(city_demand_hulls)} cities")

In [ ]:
# Create HTML maps for all cities
os.makedirs('demand_maps', exist_ok=True)

def create_city_map(city, hull):
    """Create folium map for a city showing demand hull."""
    centroid = hull.centroid
    m = folium.Map(location=[centroid.y, centroid.x], zoom_start=13)
    
    def add_polygon(poly):
        coords = [(lat, lon) for lon, lat in poly.exterior.coords]
        folium.Polygon(locations=coords, color='blue', weight=2, fill=True, fill_opacity=0.3).add_to(m)
        for interior in poly.interiors:
            hole_coords = [(lat, lon) for lon, lat in interior.coords]
            folium.Polygon(locations=hole_coords, color='red', weight=3, fill=True, fill_color='red', fill_opacity=0.5).add_to(m)
    
    if hull.geom_type == 'Polygon':
        add_polygon(hull)
    elif hull.geom_type == 'MultiPolygon':
        for poly in hull.geoms:
            add_polygon(poly)
    return m

for city, hull in tqdm(city_demand_hulls.items(), desc="Creating maps"):
    m = create_city_map(city, hull)
    safe_name = sanitize_city_name(city)
    m.save(f'demand_maps/{safe_name}.html')

<cell_type>markdown</cell_type>## Build Availability Model

For each scooter, track when and where it was available:
- **Intervals**: (start_time, end_time, lat, lon) - new interval when scooter moves >100m

In [ ]:
def build_availability_intervals(events_df, city, min_move_meters=100):
    """Build availability intervals. New interval when scooter moves >100m."""
    city_events = events_df[events_df['city'] == city].sort_values(['vehicle_id', 'timestamp'])
    intervals = []
    
    for vid, veh_events in city_events.groupby('vehicle_id'):
        current = None
        for _, e in veh_events.iterrows():
            if current is None:
                current = {'vehicle_id': vid, 'start_time': e['timestamp'], 'lat': e['lat'], 'lon': e['lon']}
            else:
                dist_m = np.sqrt((e['lat'] - current['lat'])**2 + (e['lon'] - current['lon'])**2) * 111000
                if dist_m > min_move_meters:
                    current['end_time'] = e['timestamp']
                    intervals.append(current)
                    current = {'vehicle_id': vid, 'start_time': e['timestamp'], 'lat': e['lat'], 'lon': e['lon']}
        if current:
            current['end_time'] = veh_events['timestamp'].iloc[-1]
            intervals.append(current)
    
    if not intervals:
        return pd.DataFrame(columns=['vehicle_id', 'start_time', 'end_time', 'lat', 'lon', 'duration_min'])
    
    df = pd.DataFrame(intervals)
    df['duration_min'] = (df['end_time'] - df['start_time']).dt.total_seconds() / 60
    return df[df['duration_min'] >= 1]

# Build for all cities
city_availability = {}
for city in tqdm(city_departures.keys(), desc="Building availability intervals"):
    city_availability[city] = build_availability_intervals(events, city)

print(f"Availability intervals built for {len(city_availability)} cities")

<cell_type>markdown</cell_type>## Calculate Average Distance to Nearest Scooter

Pipeline:
1. **Time sampling**: Every hour across the data range
2. **For each sample time**: Filter to scooters actually available at that exact moment
3. **Build BallTree**: From positions of currently-available scooters only
4. **Location sampling**: 5,000 random points from city demand hull
5. **Query**: Find nearest available scooter for each point

In [ ]:
import time

def add_noise(points, max_meters=200):
    """Add random noise up to max_meters to each point."""
    n = len(points)
    angles = np.random.uniform(0, 2*np.pi, n)
    distances = np.random.uniform(0, max_meters, n)
    dlat = (distances * np.cos(angles)) / 111000
    dlon = (distances * np.sin(angles)) / 111000
    noisy = points.copy()
    noisy[:, 0] += dlat
    noisy[:, 1] += dlon
    return noisy

def uniform_sample(points, grid_size_m=50):
    """Keep one random point per grid cell for uniform spatial distribution."""
    grid_deg = grid_size_m / 111000
    cell_x = (points[:, 1] / grid_deg).astype(int)
    cell_y = (points[:, 0] / grid_deg).astype(int)
    cell_ids = cell_x * 1000000 + cell_y
    
    df = pd.DataFrame({'lat': points[:, 0], 'lon': points[:, 1], 'cell': cell_ids})
    uniform = df.groupby('cell').apply(lambda x: x.sample(1)).reset_index(drop=True)
    return uniform[['lat', 'lon']].values

# Pre-compute uniform sample points for each city
city_sample_points = {}
for city in tqdm(city_availability.keys(), desc="Uniform sampling"):
    departures_raw = city_departures[city][['lat', 'lon']].values
    city_sample_points[city] = uniform_sample(departures_raw, grid_size_m=50)

In [ ]:
def calculate_median_distance(city, sample_interval_hours=1, show_progress=False):
    """
    Calculate median distance to nearest scooter.
    Uses uniform sample points + 200m noise for unbiased spatial coverage.
    """
    if city not in city_availability or city not in city_sample_points:
        return np.nan
    
    intervals = city_availability[city]
    sample_pts = city_sample_points[city]
    
    if len(intervals) == 0 or len(sample_pts) == 0:
        return np.nan
    
    # Pre-convert intervals to numpy for fast filtering
    start_times = intervals['start_time'].values
    end_times = intervals['end_time'].values
    lats = intervals['lat'].values
    lons = intervals['lon'].values
    
    # Get time range
    min_time, max_time = intervals['start_time'].min(), intervals['end_time'].max()
    sample_times = pd.date_range(min_time, max_time, freq=f'{sample_interval_hours}h')
    
    all_distances = []
    iterator = tqdm(sample_times, desc=f"{city}", leave=False) if show_progress else sample_times
    for t in iterator:
        t_np = np.datetime64(t)
        
        # Fast numpy filtering
        mask = (start_times <= t_np) & (end_times > t_np)
        if mask.sum() < 5:
            continue
        
        # Build BallTree for active scooters
        coords_rad = np.radians(np.column_stack([lats[mask], lons[mask]]))
        tree = BallTree(coords_rad, metric='haversine')
        
        # Add noise to sample points for each timestep
        noisy_pts = add_noise(sample_pts, max_meters=200)
        distances, _ = tree.query(np.radians(noisy_pts), k=1)
        all_distances.extend(distances.flatten() * 6371000)  # radians to meters
    
    return np.median(all_distances) if all_distances else np.nan

# Quick test
t0 = time.time()
test_dist = calculate_median_distance('Stuttgart', show_progress=True)
print(f"\nStuttgart: {test_dist:.0f}m median distance ({time.time()-t0:.1f}s)")

In [ ]:
# Calculate for all cities with checkpointing
DISTANCE_CHECKPOINT = 'checkpoints/availability_results.pkl'

if os.path.exists(DISTANCE_CHECKPOINT):
    with open(DISTANCE_CHECKPOINT, 'rb') as f:
        existing_results = pickle.load(f)
else:
    existing_results = {}

cities_to_process = [c for c in city_availability.keys() if c not in existing_results]

for city in tqdm(cities_to_process, desc="Cities"):
    median_dist = calculate_median_distance(city, show_progress=True)
    existing_results[city] = median_dist
    with open(DISTANCE_CHECKPOINT, 'wb') as f:
        pickle.dump(existing_results, f)

availability_results = [{'city': city, 'median_distance_m': median_dist} 
                        for city, median_dist in existing_results.items()]
availability_df = pd.DataFrame(availability_results)

print(availability_df.sort_values('median_distance_m').to_string())

## Load Trip Data for Rides per 1k Inhabitants

In [ ]:
# Load trip data
trips = pd.read_parquet('pt_comparison_all_trips.parquet')
trips = filter_known_issues(trips, normalize_city_func=normalize_city)

trips['date'] = pd.to_datetime(trips['d_time']).dt.date
n_days = trips['date'].nunique()

daily_rides = trips.groupby('city_name').size() / n_days
daily_rides = daily_rides.reset_index()
daily_rides.columns = ['city', 'mean_daily_rides']

daily_rides['population'] = daily_rides['city'].map(CITY_POPULATIONS)
daily_rides['rides_per_1k'] = (daily_rides['mean_daily_rides'] / daily_rides['population']) * 1000

## Correlate Availability with Usage

In [ ]:
# Merge availability with usage data
merged = availability_df.merge(daily_rides, on='city', how='inner')
merged = merged[merged['rides_per_1k'].notna() & merged['median_distance_m'].notna()]

print(f"Cities with both availability and usage data: {len(merged)}")

In [ ]:
plt.rcParams.update(bundles.icml2024(column="half", nrows=1, ncols=1))

fig, ax = plt.subplots()

ax.scatter(merged['median_distance_m'], merged['rides_per_1k'], s=50, c=[rgb.tue_blue])

# Regression line
z = np.polyfit(merged['median_distance_m'], merged['rides_per_1k'], 1)
p = np.poly1d(z)
x_line = np.linspace(merged['median_distance_m'].min(), merged['median_distance_m'].max(), 100)
ax.plot(x_line, p(x_line), '--', color=rgb.tue_red, label='Trend')

ax.set_xlabel('Median Distance to Nearest Scooter (m)')
ax.set_ylabel('Daily Rides per 1,000 Inhabitants')
ax.set_xlim(0, 3000)

texts = []
for _, row in merged.iterrows():
    texts.append(ax.text(row['median_distance_m'], row['rides_per_1k'], row['city'], fontsize='small'))

adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='-', color='gray', lw=0.5))

plt.savefig("availability_vs_usage.pdf")
plt.show()

# Correlation statistics
pearson_r = merged['median_distance_m'].corr(merged['rides_per_1k'])
spearman_rho, spearman_p = stats.spearmanr(merged['median_distance_m'], merged['rides_per_1k'])

print(f"Pearson r = {pearson_r:.3f}, Spearman rho = {spearman_rho:.3f} (p = {spearman_p:.4f})")